In [5]:
from numpy.core.arrayprint import DatetimeFormat
import pandas as pd
import numpy as np

from pathlib import Path

def reindex_dates_to_hour():
    file_url = "gridwatch 2017-2018 5mins.csv"

    df = pd.read_csv(file_url, engine='python', parse_dates=True, index_col=0)

    df.index = pd.to_datetime(df.index)

    df = df[~df.index.duplicated(keep='first')]

    df = df.resample('H').sum()
  
    df.to_csv("output.csv",header=["demand"])

# reindex_dates_to_hour()


''' county namesfdf'''
# town_names = ["Wiltshire-Lyneham", "Gloucestershire-Little Rissington", "Cornwall-Camborne", "Devon-Exeter", "Cornwall-Cardinham Bodmin", "Somerset-Liscombe", "Somerset-Yeovilton",
#               "Dorset-Hurn", "Hampshire-Odiham", "Surrey-Wisley", "Kent-East Malling", "Kent-Manston", "Greater London-Heathrow", "Greater London-Kew Gardens", "Greater London-Kenley"]

# town_names = ["Wiltshire-Lyneham", "Gloucestershire-Little Rissington", "Cornwall-Camborne", "Devon-Exeter", "Cornwall-Cardinham Bodmin", "Somerset-Liscombe", "Somerset-Yeovilton",
#               "Dorset-Hurn", "Hampshire-Odiham", "Surrey-Wisley", "Kent-East Malling", "Kent-London", "Greater London-Heathrow", "Greater London-Kew Gardens", "Greater London-Kenley"]


# town_names = ["South East", "South West", "Outer London" ]

town_names_df=pd.read_csv("../../bulk_downloader/scripts/IMPORT_SUPPLY_DEMAND_MAP.csv", index_col=None)

town_names=list(town_names_df.loc[0])[1:]
def get_counties(town_names,df):

    local_auth_list = df['Local Authority'].tolist()

    combined = '\t'.join(local_auth_list)
    indexes = []

    for i in range(len(town_names)):
        town = town_names[i]
        for j in range(len(local_auth_list)):
            county = local_auth_list[j]
            town = town.replace('-', ' ')
            town_tokens = town.split(' ')
            for t in town_tokens:
                if (t in county):
                    # print(town, " fdsf ", county, i, j)
                    indexes.append(j)
                    break
            
            else:

                continue  # only executed if the inner loop did NOT break

            break  #
    return indexes


def get_counties_match_exact(town_names,df):
    indexes=[]

    for town in town_names:

        indexes.append(df[df['Local Authority'] == town].index[0])
    return indexes
    
def create_local_demand_timeseries(time_series_df,local_auth_df,place_name):
    #assert df.loc[df['Local Authority'] == "Wiltshire"].shape, (1,1)
    #assert time_series_df["demand"].sum() !=0 , True

    factor= (local_auth_df.loc[local_auth_df['Local Authority'] == place_name]["Total_GWh"])/(time_series_df["demand"].sum())

    #assert factor.shape, (12,2)

    time_series_df["demand"] = factor.values[0]* time_series_df["demand"]
    time_series_df.to_csv("local_demand_timeseries1/"+place_name+".csv",index=False)


local_auth_df = pd.read_csv("Subnational_electricity_consumption_statistics_2017.csv",)

#indexes=get_counties(town_names,local_auth_df)
indexes=get_counties_match_exact(town_names,local_auth_df)



local_auth_df=local_auth_df.loc[indexes]
local_auth_df = local_auth_df[~local_auth_df.index.duplicated(keep='first')]
local_auth_df = local_auth_df[['Local Authority', 'Total_GWh']]
# TODO use domestic, non-domestic, total seperately



time_series_df=pd.read_csv("output.csv")

Path("local_demand_timeseries1").mkdir(parents=True, exist_ok=True)

for place in local_auth_df['Local Authority'].tolist():
    create_local_demand_timeseries(time_series_df,local_auth_df,place)





In [8]:
import glob

path = './local_demand_timeseries1/' # use your path
all_files = glob.glob(path + "/*.csv")

li = []
count=0
for filename in all_files:

    # if count==0:
    #     df = pd.read_csv(filename, index_col=None)
    #     name=town_names_df.apply(lambda row: row[row == filename.split("\\")[-1].replace(".csv","")].index, axis=1)[0].values[0]
    #     df.columns=['time',name+" (MWh)"]#
    #     li.append(df)
    # else:

    df = pd.read_csv(filename, index_col=None)
    df = df.drop('timestamp', 1)

    name=town_names_df.apply(lambda row: row[row == filename.split("\\")[-1].replace(".csv","")].index, axis=1)[0].values[0]
    df.columns=[name+" (MWh)"]

    li.append(df)
    # count+=1

frame = pd.concat(li, axis=1)
frame.index.names = ['time']

frame.to_csv("compiled_local_demand_ts.csv")
frame

,argyll-in-strathclyde-region_tiree (MWh),kirkcudbrightshire_dundrennan (MWh),londonderry_magilligan (MWh),suffolk_wattisham (MWh),bedfordshire_bedford (MWh),dorset_hurn (MWh),cornwall_camborne (MWh),south-glamorgan_st-athan (MWh),avon_filton (MWh),kirkcudbrightshire_threave (MWh),...,shropshire_shawbury (MWh),warwickshire_church-lawford (MWh),devon_north-wyke (MWh),warwickshire_coleshill (MWh),cheshire_rostherne (MWh),east-sussex_herstmonceux-west-end (MWh),aberdeenshire_braemar (MWh),fife_leuchars (MWh),surrey_wisley (MWh),shetland_lerwick (MWh)
time,,,,,,,,,,,,,,,,,,,,,
0,0.114721,0.047169,0.052049,0.040496,0.072905,0.154793,0.121188,0.061667,0.185093,0.048944,...,0.086654,0.051154,0.058305,0.033873,0.143561,0.054613,0.034280,0.084109,0.060706,0.050428
1,0.114817,0.047209,0.052093,0.040530,0.072967,0.154923,0.121290,0.061719,0.185249,0.048985,...,0.086726,0.051197,0.058354,0.033901,0.143681,0.054659,0.034308,0.084180,0.060757,0.050471
2,0.111404,0.045805,0.050544,0.039325,0.070797,0.150317,0.117684,0.059884,0.179741,0.047528,...,0.084148,0.049675,0.056619,0.032893,0.139409,0.053034,0.033288,0.081677,0.058950,0.048970
3,0.105688,0.043455,0.047951,0.037307,0.067165,0.142604,0.111645,0.056811,0.170519,0.045090,...,0.079830,0.047126,0.053714,0.031205,0.132256,0.050313,0.031580,0.077486,0.055925,0.046457
4,0.101767,0.041843,0.046172,0.035923,0.064673,0.137314,0.107503,0.054704,0.164192,0.043417,...,0.076869,0.045378,0.051721,0.030048,0.127349,0.048446,0.030409,0.074612,0.053851,0.044734
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,0.142566,0.058618,0.064682,0.050324,0.090601,0.192364,0.150602,0.076635,0.230018,0.060823,...,0.107686,0.063570,0.072457,0.042094,0.178405,0.067868,0.042600,0.104524,0.075440,0.062668
8756,0.131528,0.054080,0.059674,0.046428,0.083586,0.177470,0.138942,0.070701,0.212209,0.056114,...,0.099348,0.058648,0.066847,0.038835,0.164592,0.062614,0.039301,0.096431,0.069599,0.057816
8757,0.121400,0.049915,0.055079,0.042853,0.077150,0.163805,0.128243,0.065257,0.195869,0.051793,...,0.091698,0.054132,0.061699,0.035845,0.151918,0.057792,0.036275,0.089006,0.064240,0.053364


In [7]:

town_names_df.apply(lambda row: row[row == 'Falkirk'].index, axis=1)[0].values[0]


'sutherland_altnaharra'